In [1]:
from liberate import fhe
from liberate.fhe import presets
from liberate.utils.helpers import absmax_error

# 1. Set params

In [2]:
params = presets.params["silver"]

num_of_parties = 10
print(params)

engine = fhe.ckks_engine(verbose=True, **params)

{'logN': 15, 'num_special_primes': 2, 'devices': [0], 'scale_bits': 40, 'num_scales': None}
[2023-11-20 02:05:56.001708] I have received the context:


I have received inputs:
        buffer_bit_length		= 62
        scale_bits			= 40
        logN				= 15
        N				= 32,768
        Number of special primes	= 2
        Number of scales		= 16
        Cache folder			= '/home/hanyul/.pyenv/versions/liberate/lib/python3.11/site-packages/liberate/fhe/cache/resources'
        Security bits			= 128
        Quantum security model		= post_quantum
        Security sampling distribution	= uniform
        Number of message bits		= 60
        In total I will be using '821' bits out of available maximum '829' bits.
        And is it secured?		= True
My RNS primes are [1099510054913, 1099515691009, 1099507695617, 1099516280833, 1099506515969, 1099520606209, 1099504549889, 1099523555329, 1099503894529, 1099527946241, 1099503370241, 1099529060353, 1099498258433, 1099531223041, 1099469684737, 109953200

# 2. Generate sk, cpk

In [3]:
###################################
####    generate secret key    ####
###################################
sks = [engine.create_secret_key() for _ in range(num_of_parties)]


###################################
####    generate public key    ####
###################################
pks = [engine.create_public_key(sk=sks[0])]
crs = engine.clone(pks[0]).data[1]
for sk in sks[1:]:
    pks.append(engine.multiparty_create_public_key(sk, a=crs))

##############################################
####    generate collective public key    ####
##############################################
cpk = engine.multiparty_create_collective_public_key(pks=pks)

# 3. Generate Collective Evaluation Key (cevk)

In [4]:
evks_share = [engine.create_key_switching_key(sks[0], sks[0])]
crs = engine.generate_rotation_crs(evks_share[0])

# generate each party's evk_share
for sk in sks[1:]:
    evks_share.append(engine.multiparty_create_key_switching_key(sk, sk, a=crs))

In [5]:
########################
#### add evks_share ####
########################
evk_sum = engine.multiparty_sum_evk_share(evks_share)

In [6]:
######################################
####    mult sk_i and evks_sum    ####
######################################
evk_sum_mult = [engine.multiparty_mult_evk_share_sum(evk_sum, sk) for sk in sks]

In [7]:
###################################
####    sum up evk sum mult    ####
###################################

cevk = engine.multiparty_sum_evk_share_mult(evk_sum_mult)

In [8]:
print("====="*20)
print(f"|\tfrom\t|\t\t multiparty cevk {engine.num_levels}\t\t\t\t|\tto\t|")
print("====="*20)
for level in range(engine.num_levels-4):
    amin, amax = -255, 255
    m = engine.example(amin=amin, amax=amax)
    m_r = m*m
    ct = engine.encorypt(m, cpk, level=level)
    ct_ = engine.mult(ct, ct, evk=cevk)

    pcts = [engine.multiparty_decrypt_head(ct_, sks[0])]
    for sk in sks[1:]:
        pcts.append(engine.multiparty_decrypt_partial(ct_, sk))
    m_ = engine.multiparty_decrypt_fusion(pcts, level=ct_.level)

    print(f"|\t{level:2d}\t| abs max error : {absmax_error(m_, m_r):.15e}\t|\t{level+1}\t|")
    print("====="*20)

|	from	|		 multiparty cevk 16				|	to	|
|	 0	| abs max error : 5.251673428574577e-05+5.305385275278240e-05j	|	1	|
|	 1	| abs max error : 4.791437095263973e-05+6.368057802319527e-05j	|	2	|
|	 2	| abs max error : 5.366812183638103e-05+5.742147914133966e-05j	|	3	|
|	 3	| abs max error : 6.590963312191889e-05+6.023500463925302e-05j	|	4	|
|	 4	| abs max error : 5.644744669552892e-05+5.796311597805470e-05j	|	5	|
|	 5	| abs max error : 5.967102515569422e-05+5.590888031292707e-05j	|	6	|
|	 6	| abs max error : 5.736774255638011e-05+7.757947605568916e-05j	|	7	|
|	 7	| abs max error : 5.496021731232759e-05+5.005866114515811e-05j	|	8	|
|	 8	| abs max error : 4.584088196679659e-05+6.713401671731845e-05j	|	9	|
|	 9	| abs max error : 7.109152647899464e-05+8.031859761103988e-05j	|	10	|
|	10	| abs max error : 8.494409030390671e-05+7.240792911034077e-05j	|	11	|
|	11	| abs max error : 6.169732205307810e-05+5.162700836081058e-05j	|	12	|


In [9]:
print("====="*20)
print(f"|\tfrom\t|\t\t multiparty cevk {engine.num_levels}\t\t\t\t|\tto\t|")
print("====="*20)
for level in range(engine.num_levels-2):
    amin, amax = -10, 10
    m = engine.example(amin=amin, amax=amax)
    m_r = m*m*m*m
    ct = engine.encorypt(m, cpk, level=level)
    ct_ = engine.mult(ct, ct, evk=cevk)
    ct_ = engine.mult(ct_, ct_, evk=cevk)

    pcts = [engine.multiparty_decrypt_head(ct_, sks[0])]
    for sk in sks[1:]:
        pcts.append(engine.multiparty_decrypt_partial(ct_, sk))
    m_ = engine.multiparty_decrypt_fusion(pcts, level=ct_.level)

    print(f"|\t{level:2d}\t| abs max error : {absmax_error(m_, m_r):.15e}\t|\t{ct_.level}\t|")
    print("====="*20)

|	from	|		 multiparty cevk 16				|	to	|
|	 0	| abs max error : 8.379804057767615e-04+6.419643473236647e-04j	|	2	|
|	 1	| abs max error : 6.829361882410012e-04+7.481787697543041e-04j	|	3	|
|	 2	| abs max error : 7.354890913120471e-04+1.435421461337683e-03j	|	4	|
|	 3	| abs max error : 6.530667833430925e-04+6.227328467502957e-04j	|	5	|
|	 4	| abs max error : 8.230251332861371e-04+1.020490788960160e-03j	|	6	|
|	 5	| abs max error : 5.370494982344098e-04+7.487396105716471e-04j	|	7	|
|	 6	| abs max error : 7.269913658092264e-04+6.819810532761039e-04j	|	8	|
|	 7	| abs max error : 5.843273902428336e-04+6.542416995216627e-04j	|	9	|
|	 8	| abs max error : 7.579360317322426e-04+5.360916848076158e-04j	|	10	|
|	 9	| abs max error : 7.405066025967244e-04+6.557972155860625e-04j	|	11	|
|	10	| abs max error : 7.289554341696203e-04+5.919641971559031e-04j	|	12	|
|	11	| abs max error : 7.807071560819168e-04+7.009781235183254e-04j	|	13	|
|	12	| abs max error : 6.832058497820981e-04+6.227795997801877e-04j	